In [ ]:
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


Mounted at /gdrive
/gdrive


In [ ]:
trainfile = r'/gdrive/My Drive/ASSIGNMENT2/pb_train.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/gdrive/My Drive/ASSIGNMENT2/pb_test.csv'
testData = pd.read_csv(testfile)  #creates a dataframe


print(trainData.shape)
print(testData.shape)

(4521, 17)
(45211, 17)


In [ ]:
trainData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


In [ ]:
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [ ]:
TrainCols = list(trainData.columns.values)
TestCols = list(testData.columns.values)
print(TrainCols)
print(TestCols)

['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']


In [ ]:
# Seperate Target column from Train Data
Xtrain = trainData[TrainCols[0:len(TrainCols)-1]].copy()
Ytrain = trainData[['y']].copy()
print("Train Set shape:")
print(Xtrain.shape)
print(Ytrain.shape)
Xtest = testData[TestCols[0:len(TestCols)-1]].copy()
Ytest = testData[['y']].copy()
print("Test Set shape:")
print(Xtest.shape)
print(Ytest.shape)

Train Set shape:
(4521, 16)
(4521, 1)
Test Set shape:
(45211, 16)
(45211, 1)


In [ ]:
categoricalFeatures = ["job", "marital", "education", "default", "housing", "loan", "contact", "poutcome"]
print(categoricalFeatures)

categoricalFeatures1 = ["job", "marital","month", "education", "default", "housing", "loan", "contact", "poutcome"]
print(categoricalFeatures1)

['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
['job', 'marital', 'month', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']


In [ ]:
# OneHotEncoding on Train (fit & transform)
# OneHotEncoding is to be done on Categorical variables.
ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
#Xcat = pd.DataFrame(ohe.fit_transform(Xtrain[categoricalFeatures]),columns=ohe.get_feature_names(),index=Xtrain.index)
#Xtrain = pd.concat([Xtrain,Xcat],axis=1)
#Xtrain.drop(labels=categoricalFeatures,axis=1,inplace=True)
#Xtrain.sample(5)


ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
Xcat = pd.DataFrame(ohe.fit_transform(Xtrain[categoricalFeatures1]),columns=ohe.get_feature_names(),index=Xtrain.index)
Xtrain = pd.concat([Xtrain,Xcat],axis=1)
Xtrain.drop(labels=categoricalFeatures1,axis=1,inplace=True)
Xtrain.sample(5)

,age,balance,day,duration,campaign,pdays,previous,x0_admin.,x0_blue-collar,x0_entrepreneur,...,x5_yes,x6_no,x6_yes,x7_cellular,x7_telephone,x7_unknown,x8_failure,x8_other,x8_success,x8_unknown
1239,28,4229,5,205,2,-1,0,0.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2220,33,139,18,732,2,291,5,0.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4070,31,481,20,106,2,183,2,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3444,42,2974,21,187,5,-1,0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
459,43,36,26,118,1,102,2,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
# OneHotEncoding on Test (only transform)
# OneHotEncoding is to be done on Categorical variables.
Xcat = pd.DataFrame(ohe.transform(Xtest[categoricalFeatures1]),columns=ohe.get_feature_names(),index=Xtest.index)
Xtest = pd.concat([Xtest,Xcat],axis=1)
Xtest.drop(labels=categoricalFeatures1,axis=1,inplace=True)
Xtest.sample(5)

,age,balance,day,duration,campaign,pdays,previous,x0_admin.,x0_blue-collar,x0_entrepreneur,...,x5_yes,x6_no,x6_yes,x7_cellular,x7_telephone,x7_unknown,x8_failure,x8_other,x8_success,x8_unknown
8985,33,1240,5,400,2,-1,0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5001,38,-121,21,288,1,-1,0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
24843,37,0,18,173,1,179,6,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7757,47,1794,30,515,2,-1,0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5971,27,953,26,340,2,-1,0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
dt = DecisionTreeClassifier()


In [ ]:
dt.fit(Xtrain, Ytrain)


DecisionTreeClassifier()

In [ ]:
rf = RandomForestClassifier()


In [ ]:
rf.fit(Xtrain, Ytrain)

RandomForestClassifier()

In [ ]:
X_Pred = dt.predict(Xtest)
XPred = dt.predict(Xtrain)
#Model Accuracy
print("Train Accuracy:", metrics.accuracy_score(Ytrain,XPred))
print("Test Accuracy:", metrics.accuracy_score(Ytest,X_Pred))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,X_Pred))
print("Max Depth",dt.get_depth())
print("Leaf",dt.get_n_leaves())
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, X_Pred))

Train Accuracy: 1.0
Test Accuracy: 0.8832806175488266
Confusion Matrix for Decision Tree:
[[37185  2737]
 [ 2540  2749]]
Max Depth 26
Leaf 381
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

          no       0.94      0.93      0.93     39922
         yes       0.50      0.52      0.51      5289

    accuracy                           0.88     45211
   macro avg       0.72      0.73      0.72     45211
weighted avg       0.89      0.88      0.88     45211



In [ ]:
X_Pred1 = rf.predict(Xtest)
XPred1 = rf.predict(Xtrain)
#Model Accuracy
print("Train Accuracy:", metrics.accuracy_score(Ytrain,XPred1))
print("Test Accuracy:", metrics.accuracy_score(Ytest,X_Pred1))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,X_Pred1))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, X_Pred1))

Train Accuracy: 1.0
Test Accuracy: 0.9117692596934375
Confusion Matrix for Decision Tree:
[[39212   710]
 [ 3279  2010]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

          no       0.92      0.98      0.95     39922
         yes       0.74      0.38      0.50      5289

    accuracy                           0.91     45211
   macro avg       0.83      0.68      0.73     45211
weighted avg       0.90      0.91      0.90     45211



In [ ]:
#Hyperparameter tuning done for decision tree classifier

#RANDOM SEARCH--

import time
start_time = time.time()

print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(10,300,10),'max_depth': 
            range(5,30,2),'criterion':['gini','entropy']}
dt_random = RandomizedSearchCV(dt,parameters,n_iter=25,cv=5)
dt_random.fit(Xtrain, Ytrain)
grid_parm=dt_random.best_params_
print(grid_parm)
print("accuracy Score for Decision Tree:{0:6f}".
      format(dt_random.score(Xtest,Ytest)))

print("--- %s seconds ---" % (time.time() - start_time))


RandomizedSearchCV-Decision tree
{'min_samples_leaf': 20, 'max_depth': 19, 'criterion': 'gini'}
accuracy Score for Decision Tree:0.896839
--- 3.192920684814453 seconds ---


In [ ]:
#GRID SEARCH---- FOR DECISION TREE-----

import time
start_time = time.time()

print("GridSearchCV-Decision tree")
dt_grid = GridSearchCV(dt,parameters)
dt_grid.fit(Xtrain, Ytrain)
grid_parm1=dt_grid.best_params_     #getting best parameters combination 
print(grid_parm1)
print("accuracy Score for Decision Tree:{0:6f}".
      format(dt_grid.score(Xtest,Ytest)))

print("--- %s seconds ---" % (time.time() - start_time))

GridSearchCV-Decision tree
{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 30}
accuracy Score for Decision Tree:0.896220
--- 92.84692144393921 seconds ---


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
dtRand = DecisionTreeClassifier(**grid_parm)
dtGrid = DecisionTreeClassifier(**grid_parm1)

dtRand.fit(Xtrain,Ytrain)
dtRand_predict = dtRand.predict(Xtest)
dtGrid.fit(Xtrain,Ytrain)
dtGrid_predict = dtGrid.predict(Xtest)

In [ ]:
# Accuracy for Decision Tree using Random Search CV for Hyperparameter Tuning

print("Test Accuracy:", metrics.accuracy_score(Ytest,dtRand_predict))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,dtRand_predict))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, dtRand_predict))
clf_cv_score = cross_val_score(dtRand, Xtrain, Ytrain, cv=5, scoring="balanced_accuracy")
print(clf_cv_score)

Test Accuracy: 0.896839264780695
Confusion Matrix for Decision Tree:
[[38656  1266]
 [ 3398  1891]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

          no       0.92      0.97      0.94     39922
         yes       0.60      0.36      0.45      5289

    accuracy                           0.90     45211
   macro avg       0.76      0.66      0.70     45211
weighted avg       0.88      0.90      0.89     45211

[0.64315476 0.62567308 0.64072115 0.64490385 0.65514423]


In [ ]:
# Accuracy for Decision Tree using Grid Search for Hyperparameter Tuning

print("Test Accuracy:", metrics.accuracy_score(Ytest,dtGrid_predict))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,dtGrid_predict))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, dtGrid_predict))

Test Accuracy: 0.8962199464732035
Confusion Matrix for Decision Tree:
[[38959   963]
 [ 3729  1560]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

          no       0.91      0.98      0.94     39922
         yes       0.62      0.29      0.40      5289

    accuracy                           0.90     45211
   macro avg       0.77      0.64      0.67     45211
weighted avg       0.88      0.90      0.88     45211



Random Forest: Random & Grid Search

In [ ]:
#Hyperparameter tuning done for random forest classifier

#RANDOM SEARCH---------------------

import time
start_time = time.time()

print("RandomizedSearchCV-Random forest")
rand_parameters={'min_samples_leaf' : range(10,100,10),'max_depth': 
            range(1,10,2),'max_features':[10,20,30],'n_estimators':[20,30,40]}
rf_random = RandomizedSearchCV(rf,rand_parameters,n_iter=25,cv=5)
rf_random.fit(Xtrain, Ytrain)
grid_parm=rf_random.best_params_
print(grid_parm)
print("accuracy Score for Decision Tree:{0:6f}".
      format(rf_random.score(Xtest,Ytest)))

print("--- %s seconds ---" % (time.time() - start_time))

RandomizedSearchCV-Random forest
{'n_estimators': 20, 'min_samples_leaf': 10, 'max_features': 20, 'max_depth': 9}
accuracy Score for Decision Tree:0.902701
--- 16.989595413208008 seconds ---


In [ ]:
#GRID SEARCH for RandomForestClassifer 

import time
start_time = time.time()

print("GridSearchCV-Random Forest")
rf_grid = GridSearchCV(rf,rand_parameters)
rf_grid.fit(Xtrain, Ytrain)
grid_parm1=rf_grid.best_params_
print(grid_parm1)
print("accuracy Score for Random Forest:{0:6f}".
      format(rf_grid.score(Xtest,Ytest)))

print("--- %s seconds ---" % (time.time() - start_time))

GridSearchCV-Random Forest
{'max_depth': 5, 'max_features': 20, 'min_samples_leaf': 10, 'n_estimators': 30}
accuracy Score for Decision Tree:0.899936
--- 299.11602210998535 seconds ---


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier 
rfRand = RandomForestClassifier(**grid_parm)
rfGrid = RandomForestClassifier(**grid_parm1)

rfRand.fit(Xtrain,Ytrain)
rfRand_predict = rfRand.predict(Xtest)
rfGrid.fit(Xtrain,Ytrain)
rfGrid_predict = rfGrid.predict(Xtest)

In [ ]:
# Accuracy for Random Forest using Random Search CV for Hyperparameter Tuning

print("Test Accuracy:", metrics.accuracy_score(Ytest,rfRand_predict))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(Ytest,rfRand_predict))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, rfRand_predict))
clf_cv_score = cross_val_score(rfRand, Xtrain, Ytrain, cv=5, scoring="balanced_accuracy")
print(clf_cv_score)

Test Accuracy: 0.9016611001747362
Confusion Matrix for Random Forest:
[[39072   850]
 [ 3596  1693]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

          no       0.92      0.98      0.95     39922
         yes       0.67      0.32      0.43      5289

    accuracy                           0.90     45211
   macro avg       0.79      0.65      0.69     45211
weighted avg       0.89      0.90      0.89     45211

[0.63449405 0.65745192 0.62985577 0.64802885 0.63298077]


In [ ]:
# Accuracy for Random Forest using Grid Search for Hyperparameter Tuning

print("Test Accuracy:", metrics.accuracy_score(Ytest,rfGrid_predict))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,rfGrid_predict))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, rfGrid_predict))

Test Accuracy: 0.901130255911172
Confusion Matrix for Decision Tree:
[[39240   682]
 [ 3788  1501]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

          no       0.91      0.98      0.95     39922
         yes       0.69      0.28      0.40      5289

    accuracy                           0.90     45211
   macro avg       0.80      0.63      0.67     45211
weighted avg       0.89      0.90      0.88     45211

